<a href="https://colab.research.google.com/github/Coffinbrain/lessons/blob/main/TRANSFORMER_RATING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB

In [12]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('/content/IMDB_Movies_2021.db')

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT AUTHOR,TITLE,REVIEW,RATING FROM REVIEWS", conn)

df = df.dropna(subset=['RATING'])
# Ensure the rating is an integer and the review text is a string
df['RATING'] = df['RATING'].astype(float)
df['REVIEW'] = df['REVIEW'].astype(str)
df['RATING'] = df['RATING'].round().astype(int)
# Close the database connection
conn.close()


In [16]:
print(df)

                    AUTHOR                                              TITLE  \
0          margarida-44311                                          Not Bad\n   
1                 joemay-2   What are all the bad reviews about is it a wo...   
2                     nebk                            Great White=Jaws Lite\n   
3                kuarinofu                     Bare-bones killer shark film\n   
4     Horror_Flick_Fanatic                Terrible story, dialogue, and CGI\n   
...                    ...                                                ...   
5445         suryajijvania                                       More Parts\n   
5446      shishirkmr-82243                           It's a fantastic movie\n   
5447        moizsyed-07601                            Awesome out standing!\n   
5448        samun_shrestha                                             EPIC\n   
5449       mmuradali-65680                      The best DC movie till date\n   

                           

In [19]:
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel

train_df, test_df = train_test_split(df, test_size=0.3)

# Configure the model
model_args = {
    "num_train_epochs": 3,
    "learning_rate": 1e-5,
    "overwrite_output_dir": True,
    "no_save": True,  # Useful for training in Colab to avoid unnecessary saves
}

# Initialize the model
model = ClassificationModel(
    "roberta", "roberta-base", num_labels=10, args=model_args, use_cuda=True
)

# Train the model
# Adjust the ratings in the DataFrame to start from 0
#train_df['RATING'] = train_df['RATING'] - 1

# Then, pass the DataFrame to the train_model function
model.train_model(train_df[['REVIEW', 'RATING']])


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from sklearn.metrics import classification_report
#test_df['RATING'] = test_df['RATING']  1
predictions, raw_outputs = model.predict(test_df['REVIEW'].tolist())
# Compute metrics manually
print(classification_report(test_df['RATING'], predictions, digits=4))
